In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_excel('corona_lab_tests_ver002.xlsx')        # from https://data.gov.il/dataset/covid-19
df['result'] = df['corona_result'].map({'חיובי' : 'pos',
                                        'שלילי' : 'neg',
                                        'בעבודה' : 'wip',
                                        'לא ודאי' : 'uncertain'
                                           })

In [ ]:
# distribution of test results by lab
m = df
p=m.groupby(['lab_id', 'result']).count().unstack()
p['test_date'].plot(kind='barh', stacked=True)
#.plot(kind='bar')
plt.show()

In [ ]:
# number of tests within each lab by date
m = df
# filter by labs with enough total tests
lab_counts=m['lab_id'].value_counts()
biglabs=lab_counts[lab_counts > 7000].index.tolist()
m = m[m['lab_id'].isin(biglabs)]
p=m.groupby(['test_date', 'lab_id']).count().unstack()
p['result_date'].plot(linewidth=2)
plt.show()

In [ ]:
# mean days to test result
m = df
m['result_date'] = pd.to_datetime(m['result_date'])
m['test_date'] = pd.to_datetime(m['test_date'])
m['test duration'] = (m['result_date'].values - m['test_date'].values).astype('timedelta64[D]').astype('float')
lab_counts=m['lab_id'].value_counts()
biglabs=lab_counts[lab_counts > 10000].index.tolist()
m = m[m['lab_id'].isin(biglabs)]

p=m.groupby(['test_date', 'lab_id']).mean().unstack()
#p=m.groupby(['test_date']).mean().unstack()

ax = p['test duration'].plot(linewidth=2)
ax.set_ylabel('mean days to test result')
ax.set_xlabel('test date')
plt.show()